In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
from src.models.components.mlp import MyMLP

import torch
import torchsde
from torch import vmap
from torchdiffeq import odeint

from src.energies.gmm_energy import GMM
from fab.target_distributions import gmm

from src.energies.base_prior import Prior, MeanFreePrior

from src.models.components.clipper import Clipper

from torch.func import hessian
from math import sqrt
from src.utils.data_utils import remove_mean
from math import sqrt, log

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

### Target Distribution

In [ ]:
import torch


def product_of_gaussians(mu1, sigma1, mu2, sigma2, log_weights):
    EPS = 1e-6

    var1 = sigma1**2
    var2 = sigma2**2

    denom = var1 + var2
    mu_prod = (mu1 * var2 + mu2 * var1) / denom
    var_prod = (var1 * var2) / denom
    std_prod = var_prod**0.5

    diff = mu1 - mu2

    log_weights = (
        log_weights
        - 0.5 * torch.log(2 * np.pi * torch.prod(denom))
        + torch.sum(-(diff**2) / (2 * denom), dim=-1)
    )

    return mu_prod, std_prod, log_weights


def gmm_product(gmm1, gmm2):
    means_1 = gmm1.gmm.locs
    scale_trils_1 = gmm1.gmm.scale_trils
    weights_1 = gmm1.gmm.cat_probs

    means_2 = gmm2.gmm.locs
    scale_trils_2 = gmm2.gmm.scale_trils
    weights_2 = gmm2.gmm.cat_probs

    K_1 = means_1.shape[0]
    K_2 = means_2.shape[0]

    new_weights = []
    new_means = []
    new_stds = []

    for i in range(K_1):
        for j in range(K_2):
            mu1, sigma1 = means_1[i], torch.diagonal(scale_trils_1[i], dim1=-2, dim2=-1)
            mu2, sigma2 = means_2[j], torch.diagonal(scale_trils_2[i], dim1=-2, dim2=-1)
            log_weights1, log_weights2 = weights_1[i], weights_2[j]

            # Product of two Gaussians
            mu_prod, std_prod, z = product_of_gaussians(
                mu1, sigma1, mu2, sigma2, log_weights1 + log_weights2
            )

            # New weight
            new_weights.append(z)
            new_means.append(mu_prod)
            new_stds.append(std_prod)

    # Stack results into tensors
    new_weights = torch.stack(new_weights).to(gmm1.device)
    new_means = torch.stack(new_means).to(gmm1.device)
    new_stds = torch.stack(new_stds).to(gmm1.device)

    # drop modes with small logprob
    mask = torch.softmax(new_weights, dim=-1) > 1e-4
    new_weights = new_weights[mask]
    new_means = new_means[mask]
    new_stds = new_stds[mask]

    product_gmm = GMM(
        dimensionality=gmm1.dimensionality,
        n_mixes=new_means.shape[0],
        loc_scaling=1.0,
        log_var_scaling=1.0,
        mean=new_means,
        scale=new_stds,
        cat_probs=new_weights,
        device=device,
        should_unnormalize=True,
    )

    return product_gmm

In [ ]:
dimensionality = 2
temperature = 1.0
annealed_temperature = 1 / 3
inverse_temperature = temperature / annealed_temperature  # beta in the paper

target = GMM(
    dimensionality=dimensionality,
    n_mixes=40,
    loc_scaling=40.0,
    log_var_scaling=2.0,
    device=device,
    should_unnormalize=True,
)

annealed_target = gmm_product(target, gmm_product(target, target))
# annealed_target = gmm_product(target, target)
# target = target_temp2 #gmm_product(target_temp1, target_temp2)

In [ ]:
samples = {}
energies = {}
choices = {}

In [ ]:
annealed_target.gmm.locs.shape

In [ ]:
# mask= (torch.softmax(annealed_target.gmm.cat_probs, dim=-1)>1e-3)
# masked_means = annealed_target.gmm.locs[mask]

In [ ]:
a = target.sample_test_set(10000)
b = annealed_target.sample_test_set(10000)

plt.scatter(a[:, 0].detach().cpu(), a[:, 1].detach().cpu(), label="Original", alpha=0.5, s=5)
plt.scatter(b[:, 0].detach().cpu(), b[:, 1].detach().cpu(), label="Product", alpha=0.5, s=5)
plt.scatter(
    annealed_target.gmm.locs[:, 0].cpu(), annealed_target.gmm.locs[:, 1].cpu(), label="Means", s=20
)
plt.legend()

In [ ]:
target.get_dataset_fig(a, title="Temp 1")
plt.show()

# title is annealed temperature up to 3 decimal points
annealed_target.get_dataset_fig(b, title=f"Temp {annealed_temperature:.3f}")
plt.show()

## Functions

### Utils

In [ ]:
class VPSchedule:
    def __init__(self, beta_min, beta_max):
        self.beta_min = beta_min
        self.beta_max = beta_max

    def beta(self, t):
        # beta = -2 * d/dt log_alpha
        return self.beta_min + (self.beta_max - self.beta_min) * t

    def log_mean_coeff(self, t):  # log_alpha
        return -0.25 * t**2 * (self.beta_max - self.beta_min) - 0.5 * t * self.beta_min

    # ddt log_mean_coeff = -0.5 * t * (beta_max - beta_min) - 0.5 * beta_min

    def h(self, t):  # variance
        return 1.0 - torch.exp(2.0 * self.log_mean_coeff(t))

    def log_mean(self, t):
        return torch.exp(self.log_mean_coeff(t))

In [ ]:
def compute_laplacian(model, target, t, xt, noise_schedule):
    def func_wrap(t, xt):
        return model(target, xt.unsqueeze(0), t.unsqueeze(0), noise_schedule).squeeze()

    # Calculate the Hessian matrix of the model output with respect to the input
    hessian_matrix = vmap(torch.func.jacrev(func_wrap, argnums=1))(t, xt)

    # Calculate the Laplacian as the trace of the Hessian matrix
    laplacian = hessian_matrix.diagonal(offset=0, dim1=-2, dim2=-1).sum(dim=-1)
    return laplacian.detach()

In [ ]:
def true_Ut(target, x, t, noise_schedule, ve=True):
    h_t = noise_schedule.h(t).to(device)
    mean_coeff = None
    if not ve:
        mean_coeff = noise_schedule.log_mean(t).to(device)
    target.gmm.convolve(
        h_t * (target.data_normalization_factor**2), t, var_exploding=ve, mean_coeff=mean_coeff
    )
    energy = target(x)
    target.gmm.reset()
    return energy

In [ ]:
def true_nabla_Ut(target, x, t, noise_schedule, ve=True):
    nabla_Ut = torch.func.jacrev(true_Ut, argnums=1)(target, x, t, noise_schedule, ve)
    return nabla_Ut

In [ ]:
from scipy.stats import qmc

sampler = qmc.Sobol(d=1, scramble=False)


def sample_cat(bs, next_u, logits):
    # u, next_u = sample_uniform(bs, next_u)
    u = sampler.random(bs).squeeze()
    bins = torch.cumsum(torch.softmax(logits, dim=-1), dim=-1)
    ids = np.digitize(u, bins.cpu())
    ids = torch.tensor(ids, dtype=torch.long).to(device)
    return ids, next_u


def sample_cat_sys(bs, logits):
    u = torch.rand(size=(1,))
    u = (u + 1 / bs * torch.arange(bs)) % 1
    bins = torch.cumsum(torch.softmax(logits, dim=-1), dim=-1)
    ids = np.digitize(u, bins.cpu(), right=True)

    ids[ids == logits.shape[-1]] = ids[ids == logits.shape[-1]] - 1
    # print number of unique ids
    # ids = torch.tensor(ids, dtype=torch.long).to(device)
    # print(len(torch.unique(ids)))
    return ids, None


def sample_birth_death_clocks(
    bs, accum_birth, accum_death, thresh_times, reset_transition_per_index=True
):
    # Generate random keys
    death_mask = accum_death >= thresh_times
    ids = torch.arange(bs).to(device)

    # Sample candidate replacement indices according to accumulated birth weights
    if reset_transition_per_index:
        transition_probs = accum_birth / torch.sum(accum_birth, dim=-1, keepdim=True)
        transition_probs = torch.nan_to_num(transition_probs, nan=0.0)

        row_sums = torch.sum(transition_probs, dim=-1, keepdim=True)
        zero_mask = row_sums == 0.0

        # Replace zero-probability rows with uniform probabilities
        uniform_probs = torch.ones_like(transition_probs) / transition_probs.size(-1)
        transition_probs = torch.where(zero_mask, uniform_probs, transition_probs)
        replace_ids = torch.vmap(lambda x: torch.multinomial(x, 1), randomness="different")(
            transition_probs
        ).squeeze()
    else:
        transition_probs = accum_birth / torch.sum(accum_birth)
        replace_ids = torch.multinomial(transition_probs, bs, replacement=True)

    # Replace those entries chosen for killing
    ids = torch.where(death_mask, replace_ids, ids)

    # Sample new jump thresholds
    new_thresh_times = torch.distributions.Exponential(1.0).sample((bs,)).to(device)

    thresh_times = torch.where(death_mask, new_thresh_times, thresh_times)

    # Reset birth and death weights in killed indices
    if reset_transition_per_index:
        accum_birth = torch.where(
            death_mask.unsqueeze(-1), torch.zeros_like(accum_birth), accum_birth
        )
    else:
        accum_birth = torch.where(death_mask, torch.zeros_like(accum_birth), accum_birth)

    accum_death = torch.where(death_mask, torch.zeros_like(accum_death), accum_death)

    metrics = (torch.sum(death_mask),)
    return ids, accum_birth, accum_death, thresh_times, metrics

### SDEs

In [ ]:
class VEReverse_coupled(torch.nn.Module):
    noise_type = "diagonal"
    sde_type = "ito"

    def __init__(self, noise_schedule, inverse_temp=1.0, scale_diffusion=False):
        super().__init__()
        self.noise_schedule = noise_schedule
        self.inverse_temp = inverse_temp
        self.scale_diffusion = scale_diffusion

    def f(self, t, x, resampling_interval=None):
        if t.dim() == 0:
            # repeat the same time for all points if we have a scalar time
            t = t * torch.ones(x.shape[0]).to(x.device)

        with torch.enable_grad():
            x.requires_grad_(True)
            t.requires_grad_(True)

            nabla_Ut = torch.vmap(true_nabla_Ut, in_dims=(None, 0, 0, None))(
                target, x, t, self.noise_schedule
            )
            if self.scale_diffusion:
                drift_X = (
                    (self.inverse_temp + 1) / 2 * nabla_Ut * self.g(t, x).pow(2).unsqueeze(-1)
                )
            else:
                drift_X = self.inverse_temp * nabla_Ut * self.g(t, x).pow(2).unsqueeze(-1)

            drift_A = torch.zeros(x.shape[0]).to(x.device)

            if resampling_interval is None:
                return drift_X.detach(), drift_A.detach()

            drift_A = 0.5 * (
                (self.g(t, x)[:, None] * nabla_Ut).pow(2).sum(-1)
                * (self.inverse_temp - 1)
                * self.inverse_temp
            )

        return drift_X.detach(), drift_A.detach()

    def g(self, t, x):
        g = self.noise_schedule.g(t)
        return g

    def diffusion(self, t, x, diffusion_scale=1.0):
        if t.dim() == 0:
            # repeat the same time for all points if we have a scalar time
            t = t * torch.ones_like(x).to(x.device)

        if self.scale_diffusion:
            return (
                diffusion_scale
                * self.g(t, x)
                * torch.randn_like(x).to(x.device)
                / np.sqrt(self.inverse_temp)
            )
        return diffusion_scale * self.g(t, x) * torch.randn_like(x).to(x.device)

    def dlogq_dt(self, t, x, dx, dt):
        if t.dim() == 0:
            # repeat the same time for all points if we have a scalar time
            t = t * torch.ones(x.shape[0]).to(x.device)
        nabla_logq = torch.vmap(true_nabla_Ut, in_dims=(None, 0, 0, None))(
            target, x, t, self.noise_schedule
        )
        return (nabla_logq * dx).sum(-1) - 0.5 * (self.g(t, x)[:, None] * nabla_logq).pow(2).sum(
            -1
        ) * dt

### Integrators

In [ ]:
def euler_maruyama_step_coupled(
    sde,
    x,
    t,
    a,
    dt,
    step,
    resampling_interval,
    accum_birth,
    accum_death,
    clock_thresholds,
    logq,
    resampling_strategy,
    reset_transition_per_index,
):
    # Calculate drift and diffusion terms
    drift_Xt, drift_At = sde.f(t, x, resampling_interval)
    diffusion = sde.diffusion(t, x)
    dx = drift_Xt * dt + diffusion * np.sqrt(dt)

    logq_next = None
    if logq is not None:
        dlogq_dt = sde.dlogq_dt(t, x, dx, dt)
        logq_next = logq + dlogq_dt

    # Update the state
    x_next = x + dx
    a_next = a + drift_At * dt

    if resampling_interval is None or step % resampling_interval != 0:  # or step<200:

        # if step <200:
        #     a_next = torch.zeros_like(a_next)
        return x_next, a_next, logq_next, accum_birth, accum_death, clock_thresholds, x.shape[0]

    if resampling_strategy == "birth_death_clock":
        """note, dw is correctly scaled by dt"""
        dw = drift_At * dt
        avg = torch.mean(dw, axis=0, keepdims=True)

        """ accumulate thresholded weight increments (only sizing differences)"""
        accum_death = accum_death + torch.where(dw - avg < 0, avg - dw, torch.zeros_like(dw))

        bw_inc = torch.where(dw - avg > 0, dw - avg, torch.zeros_like(dw))
        if reset_transition_per_index:
            accum_birth = accum_birth + bw_inc[:, None]
        else:
            # accum_birth = accum_birth + bw_inc
            accum_birth = torch.where(dw - avg > 0, dw - avg, torch.zeros_like(dw))

        """ sampler resets accum_birth/death, clock_thresholds where necessary """
        choice, accum_birth, accum_death, clock_thresholds, metrics = sample_birth_death_clocks(
            x.shape[0], accum_birth, accum_death, clock_thresholds
        )
        x_next = x_next[choice]
        a_next = torch.zeros_like(a_next)
        choice = choice.cpu().numpy()

    elif resampling_strategy == "systematic":
        # resample based on the weights
        choice, _ = sample_cat_sys(x.shape[0], a_next)
        a_next = torch.zeros_like(a_next)
        x_next = x_next[choice]

    return (
        x_next,
        a_next,
        logq_next,
        accum_birth,
        accum_death,
        clock_thresholds,
        len(np.unique(choice)),
    )


def integrate_sde_coupled(
    sde, x0, t_span, dt, resampling_interval, logq, resampling_strategy, reset_transition_per_index
):
    times = torch.arange(t_span[0], t_span[1], dt).to(device)
    x = x0
    x0.requires_grad = True
    samples = []
    logweights = []
    logqs = []
    choices = []
    a = torch.zeros(x.shape[0]).to(device)

    if reset_transition_per_index:
        accum_birth = torch.zeros((a.shape[0], x.shape[0])).to(device)
    else:
        accum_birth = torch.zeros(a.shape[0]).to(device)
    accum_death = torch.zeros(a.shape[0]).to(device)
    clock_thresholds = torch.distributions.Exponential(1.0).sample((a.shape[0],)).to(device)
    with torch.no_grad():
        for step, t in enumerate(times):
            x, a, logq, accum_birth, accum_death, clock_thresholds, choice = (
                euler_maruyama_step_coupled(
                    sde,
                    x,
                    1 - t,
                    a,
                    dt,
                    step + 1,
                    resampling_interval,
                    accum_birth=accum_birth,
                    accum_death=accum_death,
                    clock_thresholds=clock_thresholds,
                    logq=logq,
                    resampling_strategy=resampling_strategy,
                    reset_transition_per_index=reset_transition_per_index,
                )
            )

            if target.is_molecule:
                x = remove_mean(x, target.n_particles, target.n_spatial_dim)
            samples.append(x)
            logweights.append(a)
            logqs.append(logq)
            choices.append(choice)
            # print(len(np.unique(choice.cpu().numpy())))

    logqs = torch.stack(logqs) if logq is not None else None
    return torch.stack(samples), torch.stack(logweights), logqs, np.array(choices)


def generate_samples_weighted(
    reverse_sde,
    t_span=(0, 1),
    num_integration_steps=100,
    samples=None,
    num_samples=200,
    resampling_interval=None,
    prior=None,
    compute_logq=False,
    resampling_strategy="systematic",
    reset_transition_per_index=False,
):
    if samples is None:
        if prior is None:
            raise ValueError("Either samples or prior distribution should be provided")
        samples = prior.sample(num_samples)

    dt = 1 / num_integration_steps

    samples, weights, logqs, choices = integrate_sde_coupled(
        sde=reverse_sde,
        x0=samples,
        t_span=t_span,
        dt=dt,
        resampling_interval=resampling_interval,
        logq=prior.log_prob(samples) if compute_logq else None,
        resampling_strategy=resampling_strategy,
        reset_transition_per_index=reset_transition_per_index,
    )
    if compute_logq:
        return samples[-1], weights, logqs, choices
    else:
        return samples[-1], weights, choices

## Sampling

### Variance Exploding SDE

In [ ]:
from src.models.components.noise_schedules import GeometricNoiseSchedule, QuadraticNoiseSchedule

noise_schedule = GeometricNoiseSchedule(sigma_min=0.01, sigma_max=10 * sqrt(temperature))
# noise_schedule = QuadraticNoiseSchedule(beta=10)

In [ ]:
num_int_steps = 1000
num_samples = 10000
num_seeds = 5

prior = Prior(2, device=device, scale=noise_schedule.h(1) ** 0.5)
annealed_prior = Prior(2, device=device, scale=(noise_schedule.h(1) / inverse_temperature) ** 0.5)


prior_samples = prior.sample(num_samples)
annealed_prior_samples = annealed_prior.sample(num_samples)

In [ ]:
# Test Ito density Estimation
ve_reverse_sde = VEReverse_coupled(noise_schedule, scale_diffusion=False)
samples_hightemp, _, logq, _ = generate_samples_weighted(
    ve_reverse_sde,
    t_span=(0, 1),
    num_integration_steps=num_int_steps,
    resampling_interval=None,
    num_samples=num_samples,
    samples=prior_samples,
    prior=prior,
    compute_logq=True,
)
samples_energy = target(samples_hightemp)
# target.get_dataset_fig(target.unnormalize(samples_hightemp), T=1.0, title=f"Temperature {temperature}")
samples_hightemp_unnorm = target.unnormalize(samples_hightemp).detach().cpu().numpy()
plt.scatter(
    samples_hightemp_unnorm[:, 0],
    samples_hightemp_unnorm[:, 1],
    s=3,
    label=f"SDE T={temperature}",
    alpha=0.5,
)

s = target.sample_test_set(1000).detach().cpu().numpy()
plt.scatter(s[:, 0], s[:, 1], s=3, label=f"True T={temperature}", alpha=0.5)

s = annealed_target.sample_test_set(1000).detach().cpu().numpy()
plt.scatter(s[:, 0], s[:, 1], s=3, label=f"True T={annealed_temperature}", alpha=0.5)

plt.legend()
plt.show()

plt.hist(samples_energy.detach().cpu(), bins=200)
plt.hist((logq[-1].detach().cpu()) - log(target.data_normalization_factor**2), bins=200, alpha=0.5);

In [ ]:
# Resampling without scaling diffusion
ve_reverse_sde = VEReverse_coupled(
    noise_schedule, inverse_temp=inverse_temperature, scale_diffusion=False
)
for strategy in ["birth_death_clock", "systematic"]:
    # Resampling v1
    s = []
    c = []
    for seed in range(num_seeds):
        samples_resampled, log_weights, idxs = generate_samples_weighted(
            ve_reverse_sde,
            t_span=(0, 1),
            samples=annealed_prior_samples,
            num_integration_steps=num_int_steps,
            resampling_interval=1,
            resampling_strategy=strategy,
        )
        annealed_target.get_dataset_fig(
            annealed_target.unnormalize(samples_resampled), title="resampled " + strategy
        )
        plt.show()
        # plt.plot(torch.linspace(1, 0, num_int_steps), log_weights.cpu().detach().numpy()[:, :20])
        # plt.xlabel("Integration Time")
        # plt.ylabel("log-weights")
        # plt.show()

        samples_resampled = samples_resampled.detach()
        s.append(samples_resampled)
        c.append(idxs)

    samples["resampled " + strategy] = torch.stack(s)
    choices["resampled " + strategy] = torch.tensor(c)

In [ ]:
# Resampling with scaling diffusion
ve_reverse_sde = VEReverse_coupled(
    noise_schedule, inverse_temp=inverse_temperature, scale_diffusion=True
)

for strategy in ["birth_death_clock", "systematic"]:
    s = []
    c = []
    for seed in range(num_seeds):
        samples_resampled, log_weights, idxs = generate_samples_weighted(
            ve_reverse_sde,
            t_span=(0, 1),
            samples=annealed_prior_samples,
            num_integration_steps=num_int_steps,
            resampling_interval=1,
            resampling_strategy=strategy,
        )
        annealed_target.get_dataset_fig(
            annealed_target.unnormalize(samples_resampled),
            title="resampled scale_diffusion " + strategy,
        )
        plt.show()
        # plt.plot(torch.linspace(1, 0, num_int_steps), log_weights.cpu().detach().numpy()[:, :20])
        # plt.xlabel("Integration Time")
        # plt.ylabel("dlogW")
        # plt.show()

        samples_resampled = samples_resampled.detach()
        s.append(samples_resampled)
        c.append(idxs)

    samples["resampled " + strategy + " scale_diffusion"] = torch.stack(s)
    choices["resampled " + strategy + " scale_diffusion"] = torch.tensor(c)

In [ ]:
# plt.plot(torch.linspace(1, 0, num_int_steps), choices["resampled systematic"][0], label="Systematic seed 0", alpha=0.5)
# plt.plot(torch.linspace(1, 0, num_int_steps), choices["resampled systematic"][1], label="Systematic seed 1", alpha=0.5)
# plt.plot(torch.linspace(1, 0, num_int_steps), choices["resampled birth_death_clock"][0], label="Birth Death Clock seed 0", alpha=0.5)
# plt.plot(torch.linspace(1, 0, num_int_steps), choices["resampled birth_death_clock"][1], label="Birth Death Clock seed 1", alpha=0.5)


# plt.plot(torch.linspace(1, 0, num_int_steps), choices["resampled systematic scale_diffusion"][0], label="Systematic scale_diffusion seed 0", alpha=0.5)
# plt.plot(torch.linspace(1, 0, num_int_steps), choices["resampled systematic scale_diffusion"][1], label="Systematic scale_diffusion seed 1", alpha=0.5)

# plt.plot(torch.linspace(1, 0, num_int_steps), choices["resampled birth_death_clock scale_diffusion"][0], label="Birth Death Clock scale_diffusion seed 0", alpha=0.5)
# plt.plot(torch.linspace(1, 0, num_int_steps), choices["resampled birth_death_clock scale_diffusion"][1], label="Birth Death Clock scale_diffusion seed 1", alpha=0.5)

# plt.legend()

In [ ]:
# No Resampling without scaling diffusion
s = []
for seed in range(num_seeds):
    ve_reverse_sde = VEReverse_coupled(
        noise_schedule, inverse_temp=inverse_temperature, scale_diffusion=False
    )
    samples_not_resampled, log_weights, _ = generate_samples_weighted(
        ve_reverse_sde,
        t_span=(0, 1),
        samples=annealed_prior_samples,
        num_integration_steps=num_int_steps,
        resampling_interval=num_int_steps + 1,
    )  # num_int_steps+1
    annealed_target.get_dataset_fig(
        annealed_target.unnormalize(samples_not_resampled),
        color=log_weights.cpu()[-1],
        title="Not resampled ",
    )
    plt.show()
    # plt.plot(torch.linspace(1, 0, num_int_steps), log_weights.cpu().detach().numpy()[:, :20])
    # plt.xlabel("Integration Time")
    # plt.ylabel("log-weights")
    # plt.show()

    samples_not_resampled = samples_not_resampled.detach()
    s.append(samples_not_resampled)

samples["not_resampled"] = torch.stack(s)

# # Generate samples at high temprature
# samples_hightemp, _ = generate_samples_weighted(ve_reverse_sde, t_span=(0, 1), num_integration_steps=num_int_steps,
#                                                 resampling_interval=None, num_samples=num_samples,
#                                                 samples=prior_samples)
# samples_energy = target(samples_hightemp)
# target.get_dataset_fig(target.unnormalize(samples_hightemp), T=1.0, title=f"Temperature {temperature}")
# # samples_hightemp_unnorm = target.unnormalize(samples_hightemp).detach().cpu().numpy()
# # plt.scatter(samples_hightemp_unnorm[:, 0], samples_hightemp_unnorm[:, 1], s=3)
# plt.show()

In [ ]:
# No Resampling and scaling diffusion
s = []
for seed in range(num_seeds):
    ve_reverse_sde = VEReverse_coupled(
        noise_schedule, inverse_temp=inverse_temperature, scale_diffusion=True
    )
    samples_not_resampled, log_weights_scaled, _ = generate_samples_weighted(
        ve_reverse_sde,
        t_span=(0, 1),
        samples=annealed_prior_samples,
        num_integration_steps=num_int_steps,
        resampling_interval=num_int_steps + 1,
    )  # num_int_steps+1
    # samples_not_resampled = traj_samples[-1]
    annealed_target.get_dataset_fig(
        annealed_target.unnormalize(samples_not_resampled),
        color=log_weights.cpu()[-1],
        title="Not Resampled scale_diffusion",
    )
    plt.show()
    plt.plot(
        torch.linspace(1, 0, num_int_steps), log_weights_scaled.cpu().detach().numpy()[:, :20]
    )
    plt.xlabel("Integration Time")
    plt.ylabel("log-weights")
    plt.show()

    samples_not_resampled = samples_not_resampled.detach()
    s.append(samples_not_resampled)

samples["not_resampled scale_diffusion"] = torch.stack(s)

In [ ]:
plt.plot(torch.linspace(1, 0, num_int_steps), log_weights.mean(dim=1).cpu().detach().numpy())
plt.plot(
    torch.linspace(1, 0, num_int_steps), log_weights_scaled.mean(dim=1).cpu().detach().numpy()
)

In [ ]:
plt.plot(torch.linspace(1, 0, num_int_steps), log_weights.std(dim=1).cpu().detach().numpy())

plt.plot(torch.linspace(1, 0, num_int_steps), log_weights_scaled.std(dim=1).cpu().detach().numpy())

In [ ]:
# for i in range(5):
#     start_idx = 1
#     alpha_values = np.linspace(0.1, 1.0, start_idx)
#     plt.plot(traj_samples_not_scaled[start_idx:, i, 0].detach().cpu(),
#              traj_samples_not_scaled[start_idx:, i, 1].detach().cpu(), color='red', alpha=0.4)
#     plt.scatter(traj_samples_not_scaled[start_idx:, i, 0].detach().cpu(),
#                 traj_samples_not_scaled[start_idx:, i, 1].detach().cpu(),
#                 s=5, alpha=1)

In [ ]:
# for i in range(5):
#     start_idx = 1
#     alpha_values = np.linspace(0.1, 1.0, start_idx)
#     plt.plot(traj_samples[start_idx:, i, 0].detach().cpu(), traj_samples[start_idx:, i, 1].detach().cpu(), color='red', alpha=0.4)
#     plt.scatter(traj_samples[start_idx:, i, 0].detach().cpu(), traj_samples[start_idx:, i, 1].detach().cpu(),
#                 s=5, alpha=1)

In [ ]:
# for i in range(1000):
#     if i%100==0 or i==999:
#         plt.scatter(traj_samples[i][:, 0].detach().cpu(), traj_samples[i][:, 1].detach().cpu(),
#                     s=3, alpha=0.5, c=log_weights[i].detach().cpu())
#         plt.show()

In [ ]:
# plt.scatter(torch.linspace(1, 0, num_int_steps), log_weights.std(-1).cpu())
# plt.xlabel("Integration Time")
# plt.ylabel("Standard Deviation of dlogW")

### Metrics

In [ ]:
from src.models.components.distribution_distances import eot
from src.models.components.mmd import linear_mmd2, mix_rbf_mmd2, poly_mmd2
from src.models.components.optimal_transport import wasserstein
import ot as pot


def compute_sample_based_metrics(a, b, a_energy, b_energy):
    w1 = wasserstein(a.double(), b.double(), power=1)
    w2 = wasserstein(a.double(), b.double(), power=2)
    mmd_rbf = mix_rbf_mmd2(a, b, sigma_list=10 ** np.linspace(-2, 0, 10)).item()

    H_b, x_b, y_b = np.histogram2d(b[:, 0].cpu().numpy(), b[:, 1].cpu().numpy(), bins=200)
    H_a, x_a, y_a = np.histogram2d(a[:, 0].cpu().numpy(), a[:, 1].cpu().numpy(), bins=(x_b, y_b))
    total_var = 0.5 * np.abs(H_a / H_a.sum() - H_b / H_b.sum()).sum()

    energy_w2 = (
        pot.emd2_1d(
            a_energy,
            b_energy,
        )
        .detach()
        .cpu()
        .item()
    )
    return w1, w2, mmd_rbf, total_var, energy_w2

In [ ]:
# test_samples = target_temp1.sample_test_set(1000)
# samples_resampled = samples_resampled.detach()
# samples_not_resampled = samples_not_resampled.detach()

# samples_resampled = target_temp1.unnormalize(samples_resampled)
# samples_not_resampled = target_temp1.unnormalize(samples_not_resampled)

# plt.scatter(samples_resampled[:, 0].cpu(), samples_resampled[:, 1].cpu(), s=3)
# plt.scatter(test_samples[:, 0].cpu(), test_samples[:, 1].cpu(), s=3)
# plt.scatter(samples_not_resampled[:, 0].cpu(), samples_not_resampled[:, 1].cpu(), s=3)
# plt.show()

In [ ]:
test_samples = annealed_target.sample_test_set(num_samples)
samples["Test"] = annealed_target.normalize(test_samples).detach()


for key in samples.keys():
    e = []
    for seed in range(num_seeds):
        energy = annealed_target(samples[key][seed]).detach()
        e.append(energy)
    energies[key] = torch.stack(e)

In [ ]:
data_df = []
metrics = ["W1", "W2", "MMD", "Total Var", "Energy W2"]
for key in samples.keys():
    if key == "Test":
        continue
    for seed in range(num_seeds):
        data_df.append(
            [
                key,
                *compute_sample_based_metrics(
                    annealed_target.unnormalize(samples[key][seed]),
                    annealed_target.unnormalize(samples["Test"]),
                    energies[key][seed],
                    energies["Test"],
                ),
            ]
        )

In [ ]:
import pandas as pd

df = pd.DataFrame(data_df)
df.columns = ["Method"] + metrics

In [ ]:
# Table processing
def process_line(means, highlight, highlight_index, highlight_max, ignore_std):
    if highlight:
        if highlight_max:
            tops = set(means.groupby(highlight_index).idxmax())
        else:
            tops = set(means.groupby(highlight_index).idxmin())
    else:
        tops = set()

    def process_line(x):
        if ignore_std:
            if x.name in tops:
                return rf"\textbf{{{x['mean']:0.3f}}}"
            return rf"{x['mean']:0.3f}"
        if x.name in tops:
            return rf"\textbf{{{x['mean']:0.3f} $\pm$ {x['std']:0.3f}}}"
        return rf"{x['mean']:0.3f} $\pm$ {x['std']:0.3f}"

    return process_line


def mean_pm_std(
    data,
    index,
    columns,
    value,
    highlight=True,
    highlight_cols=True,
    highlight_max=True,
    ignore_std=False,
):
    assert len(data) > 0
    groupby = data.groupby([*index, *columns])
    means = groupby.mean()[value].rename("mean")
    stds = groupby.std()[value].rename("std")
    ddf = pd.concat([means, stds], axis=1).T
    highlight_index = columns if highlight_cols else index
    ddf = ddf.apply(process_line(means, highlight, highlight_index, highlight_max, ignore_std))
    ddf = ddf.reset_index().pivot(index=index, columns=columns)
    ddf.columns = ddf.columns.droplevel(level=0)
    return ddf

In [ ]:
df_melt = df.melt(value_vars=metrics, id_vars=["Method"], var_name="Metric").reset_index()
results = mean_pm_std(
    df_melt, index=["Method"], columns=["Metric"], value="value", highlight_max=False
)
results

In [ ]:
print(
    results.to_latex(
        float_format="{:.3f}".format,
    )
)

In [ ]:
from src.models.components.utils import sample_from_tensor

In [ ]:
# axs[0].imshow(annealed_target.get_dataset_fig(annealed_target.unnormalize(s), title=keys));

In [ ]:
from fab.utils.plotting import plot_contours, plot_marginal_pair

plotting_bounds = (-1.4 * 40, 1.4 * 40)


def get_fig(target_dist, samples, ax, title, color, fp, cmap=None):
    target_dist.gmm.to("cpu")
    plot_contours(
        target_dist.gmm.log_prob,
        bounds=plotting_bounds,
        ax=ax,
        n_contour_levels=50,
        grid_width_n_points=200,
    )

    # plot dataset samples
    plot_marginal_pair(samples, ax=ax, bounds=plotting_bounds, color=color, cmap=cmap)
    ax.set_title(title, fontproperties=fp, fontsize=20)

    ax.tick_params(labelsize=15)
    ax.set_xticks(np.arange(-40, 60, 20))
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontproperties(fp)
    ax.tick_params(labelsize=15)
    target_dist.gmm.to(device)

In [ ]:
# # save samples and energies
# torch.save(samples, "figures/gmm_samples.pt")
# torch.save(energies, "figures/gmm_energies.pt")

In [ ]:
# Get the font
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from tempfile import NamedTemporaryFile
import urllib.request as urllib

github_url = "https://github.com/google/fonts/blob/main/apache/robotomono/RobotoMono%5Bwght%5D.ttf"

url = github_url + "?raw=true"  # You want the actual file, not some html

response = urllib.urlopen(url)
f = NamedTemporaryFile(delete=False, suffix=".ttf")
f.write(response.read())
f.close()
prop = fm.FontProperties(fname=f.name)

In [ ]:
fig, axs = plt.subplots(1, 6, figsize=(30, 5), dpi=300)
# plt.xticks(fontproperties=prop)
# plt.yticks(fontproperties=prop)
# plt.xticks(fontsize=10)
# plt.yticks(fontsize=10)
# for ax in axs:

get_fig(
    target, target.sample_test_set(4000), axs[0], "Starting Disribution", color="#3B347B", fp=prop
)

titles = [
    "Ground Truth Samples",
    "Target Score",
    "Target Score + FKC",
    "Tempered Noise",
    "Tempered Noise + FKC",
]

# colors = ["Green", "Red", "Orange", "Purple", "Salmon"]
colors = ["#AEADF0", "#1D9D79", "#D96002", "#756FB3", "#FFB900"]

for i, keys in enumerate(
    [
        "Test",
        "not_resampled",
        "resampled systematic",
        "not_resampled scale_diffusion",
        "resampled systematic scale_diffusion",
    ]
):
    s = samples[keys].reshape(-1, 2)
    s = sample_from_tensor(s, 2000)
    # axs[i+1].imshow(annealed_target.get_dataset_fig(annealed_target.unnormalize(s), title=keys))
    get_fig(
        annealed_target,
        annealed_target.unnormalize(s),
        axs[i + 1],
        title=titles[i],
        color=colors[i],
        fp=prop,
    )

    # change font for
plt.savefig(f"figures/gmm_samples.png", bbox_inches="tight")
plt.show()

In [ ]:
# df.groupby("Method").mean()
# # show mean and std
# df.groupby("Method").agg([np.mean, np.std])